In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import httpx
from crawler import crawler, pattern_filter


url_queue = asyncio.Queue()
stop_crawling = asyncio.Event()
await url_queue.put("https://caseyhandmer.wordpress.com/")


import time

addable_urls = []
async with httpx.AsyncClient() as client:
    addable_urls = await crawler(
        url_queue,
        url_filter={"filter_func": pattern_filter, "kwargs": {"regex_patterns": ["https://"]}},
        client=client,
        response_queue=[],
        pause=stop_crawling,
        end=stop_crawling,
    )
    print("addable urls:", addable_urls)
    stop_crawling.set()

In [ ]:
from urllib.parse import urlparse, urlunparse
from joblib import load

addable_urls = load("cleanable_urls.joblib")

def clean_urls(urls):
    cleaned_urls = set()  # Use a set to ensure no duplicates

    # Remove fragments and trailing slashes
    for url in urls:
        parsed_url = urlparse(url)

        cleaned_url = parsed_url._replace(fragment="")

        final_url = urlunparse(cleaned_url).rstrip("/")

        cleaned_urls.add(final_url)

    return list(cleaned_urls) 

print(len(addable_urls))
cleaned_urls = clean_urls(addable_urls)
print(len(cleaned_urls))
